## PREDICTING WIND ENERGY PRODUCTION

The objetive of this project is to develop a machine learing model for estimating how much energy is going to be produced at the Sotavento experimental wind farm. To do it, we will use meteorological variables forecasted by ECMWF (http://www.ecmwf.int/) as input attributes. We have 22 variables, but, it is common practice to use the value of those variables, not just at the location of interest (Sotavento in this case), but at points in a grid around Sotavento. A 5x5 grid will be used in this case. Important to have in mind that the values at point 13 are the ones exactly Sotavento.Therefore, finally we have 550 variables (plus other 5 measuring energy, year, month, day and hour). The 22 variables are described as follows: 

- t2m: 2 metre temperature
- u10: 10 metre U wind component
- v10: 10 metre V wind component
- u100: 100 metre U wind component
- v100: 100 metre V wind component
- cape: Convective available potential energy
- flsr: Forecast logarithm of surface roughness for heat
- fsr: Forecast surface roughness
- iews: Instantaneous eastward turbulent surface stress
- inss: Instantaneous northward turbulent surface
- lai_hv: Leaf area index, high vegetation
- lai_lv: Leaf area index, low vegetation
- u10n: Neutral wind at 10 m u-component
- v10n: Neutral wind at 10 m v-component
- stl1: Soil temperature level 1
- stl2: Soil temperature level 2
- stl3: Soil temperature level 3
- stl4: Soil temperature level 4
- sp: Surface pressure
- p54.162: Vertical integral of temperature
- p59.162: Vertical integral of divergence of kinetic energy
- p55.162: Vertical integral of water vapour


First, we load the dataset.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.dummy import DummyRegressor
import time
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import mean_absolute_error

In [5]:
#wind_ava = pd.read_csv("C:\\Users\\victoria\\Escritorio\\MASTER\\Big Data Intelligence\\Assignment_1\\Big_Data_Intelligence\\wind_available.csv.gzip", compression="gzip")

In [14]:
#pongo aqui abajo mi path 

wind_ava = pd.read_csv("/Users/pameladiaz/Desktop/GitHub/Big_Data_Intelligence/wind_available.csv.gzip", compression="gzip")

## 1. Exploratory Data Analysis

It is important before starting any machine learning project to observe and understand the data we have. Therefore, during this section we will be exploring how many features and how many instances the dataset has, which variables are categorical / numerical, which features have missing values and how many, whether there are constant columns, some statistics, and whether it is a regression or classification problem. 

Finally, we will plot some of the variables in terms of the target for trying to find some relations.

### 1.1. Number of features

As previosly explained, the dataset contains 555 variables and 4747 instances.

In [ ]:
pd.set_option('display.max_rows', 10)
wind_ava

### 1.2. Variable type

All the variables we are dealing with are numerical, most of them continuous and only the ones specifiying the time when the measure was taken (year, month, day and hour) are of type integer. This leaves us with a regression problem, as the target variable 'energy' is a continuous numerical variable.

In [ ]:
pd.set_option('display.max_rows', None)
wind_ava.dtypes

### 1.3. Missing values

If any of the variables has many missing values (more than 80%) we should remove it. After checking it, we are sure none of them  has more than 80% nulls, the maximum is 20.3 % for the variable lai_hv.13 (Leaf area index, high vegetation, section 13, Sotavento). It is difficult to determine the reason most of the variables has nulls, as we are not experts on the field. We will try to solve this later on the preprocessing with imputation methods.


In [ ]:
null_percentage = wind_ava.isnull().sum()/ len(wind_ava) * 100
print(null_percentage[null_percentage > 80])
print(max(null_percentage))
null_percentage

### 1.4. Statistics

As all of our variables are numeric, we can perform a statistical analysis by giving some information about the mean, standard deviation, quantiles etc.

In [ ]:
wind_ava.describe().T


### 1.5. Constant columns

Similarly with the null values, if any column has constant values, this gives no information to the final model, so it should be removed. Taking into account that some variables have null values, we should look if the number of unique values is less or equal to 2 (the constant value and Nan). As is not the case, we don't have any column with constant values.

In [ ]:
print(wind_ava.nunique() <= 2)
#wind_ava.nunique()

### 1.6. Visualization

Let's observe first how the objective variable looks. It follows an exponential distribution, as most of the values are around the highest energy production.

In [ ]:
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10)

plt.hist(wind_ava['energy'],  bins=30, color='skyblue', edgecolor='black')
plt.title(f'Energy histogram')
plt.xlabel('Energy value')
plt.ylabel('Frequency')

# Muestra el histograma
plt.show()

We could also observe the mean enery production per year. As observed,  most of the years have similar energy production.

In [ ]:
# Calcular el promedio de energía por año
mean_energy = wind_ava.groupby('year')['energy'].mean()

# Crear un gráfico de barras
plt.figure(figsize=(10, 5))  # Ajusta el tamaño del gráfico según tus preferencias
mean_energy.plot(kind='bar', color='skyblue', edgecolor='black')

# Personalizar el gráfico (opcional)
plt.title('Energy produced per year')
plt.xlabel('Year')
plt.ylabel('Energy')
plt.grid(axis='y')

# Mostrar el gráfico de barras
plt.show()

Similarly, we could observe the mean energy production per month. This plot give us more valuable information, becuase, as it could be expected, on the months of worst weather and more wind, the energy production is highly increased.

In [ ]:
# Calcular el promedio de energía por año
mean_energy = wind_ava.groupby('month')['energy'].mean()

# Crear un gráfico de barras
plt.figure(figsize=(10, 5))  # Ajusta el tamaño del gráfico según tus preferencias
mean_energy.plot(kind='bar', color='skyblue', edgecolor='black')

# Personalizar el gráfico (opcional)
plt.title('Mean energy produced per month')
plt.xlabel('Month')
plt.ylabel('Energy')
plt.grid(axis='y')

# Mostrar el gráfico de barras
plt.show()

### 1.7. Feature transformation

Althought we have many variables, it is understandable that the variables u100 (100 metre U wind component) and v100 (100 metre V wind component) give us the most valuable information. However, the modullus is more valuable, as measures the wind speed at 100 meters high. Therefore, we would add this as a new variable to our data. 

In [15]:
## CHAT GPT
# Assuming wind_ava is your DataFrame
v_wind_columns = wind_ava.columns[wind_ava.columns.str.startswith('v100.')]
u_wind_columns = wind_ava.columns[wind_ava.columns.str.startswith('u100.')]

# Iterate through pairs of u and v columns
for u_col, v_col in zip(u_wind_columns, v_wind_columns):
    # Extract u and v components
    u_component = wind_ava[u_col]
    v_component = wind_ava[v_col]

    # Calculate wind speed
    wind_speed_col = f"wind_speed{u_col.split('.')[1]}"
    wind_ava[wind_speed_col] = np.sqrt(u_component**2 + v_component**2)

# Display the modified DataFrame with new wind speed columns
wind_ava.head()

,energy,year,month,day,hour,p54.162.1,p54.162.2,p54.162.3,p54.162.4,p54.162.5,...,wind_speed16,wind_speed17,wind_speed18,wind_speed19,wind_speed20,wind_speed21,wind_speed22,wind_speed23,wind_speed24,wind_speed25
0,402.71,2005,1,2,18,2.534970e+06,2.526864e+06,2.518754e+06,2.510648e+06,2.502537e+06,...,5.134636,NaN,NaN,NaN,4.393808,5.137268,4.948249,4.760744,4.574870,4.389944
1,696.80,2005,1,3,0,NaN,NaN,2.521184e+06,2.513088e+06,NaN,...,5.298553,NaN,4.869726,4.655077,4.440835,5.300003,5.085998,4.872320,4.658335,NaN
2,1591.15,2005,1,3,6,2.533727e+06,2.525703e+06,2.517678e+06,2.509654e+06,NaN,...,5.587899,NaN,NaN,NaN,NaN,5.529130,5.324473,5.122190,4.922326,NaN
3,1338.62,2005,1,3,12,NaN,2.526548e+06,2.518609e+06,2.510670e+06,2.502732e+06,...,4.779456,4.715311,4.655777,4.600019,4.549180,4.698198,4.634036,4.573135,4.516475,4.463542
4,562.50,2005,1,3,18,2.529543e+06,NaN,2.513702e+06,2.505782e+06,2.497861e+06,...,3.941046,NaN,NaN,3.917355,NaN,NaN,3.851946,3.843097,3.836565,3.832704


As can be observed, this new variable againts the response variable follows a linear relation, so it will give us valuable information when modelling the predictor.

In [ ]:
plt.figure(figsize=(8, 5))

# Obtain the columns defining the u and v components of the wind 
wind_speed_columns = wind_ava.columns[wind_ava.columns.str.startswith('wind_speed')]

 
plt.scatter(wind_ava['energy'], wind_ava[wind_speed_columns].mean(axis=1), c='skyblue')
plt.xlabel('Energy')
plt.ylabel('Mean wind speed')
plt.title('U and V wind components vs Energy')
plt.show()

## 2. Inner, outer evaluation and metric

### 2.1. Holdout split

After a exploration of our dataset, the next crucial step was to select the evaluation strategy. Given the considerable volume of data at our disposal, we decided to perform a holdout strategy for both inner and outer evaluations. Cross-validation was not used because it is more computationally intensive and time-consuming, particularly in the context of our substantial dataset. 

Moreover, a key factor influencing our choice is the intrinsic nature of our data, a time series with dependencies that render it non-independent and non-identically distributed (non-i.i.d.). Consequently, we have employed a partitioning strategy based on complete years. This approach ensures that each partition is not only representative of the problem at hand but also respects the temporal dependencies inherent in the data.

Visualizing the dataset through plots further informed our decision-making. Notably, for the year 2008, data is available only for January and February, whereas in 2009, we have a dataset spanning from March to December. To ensure a representative evaluation, we opted to combine the data from these two years, designating them as the test partition. The remaining years were used for training.

Later on, within our training data, we took another step and split it into two parts: train-validation and train-train. We used the data from 2007 for train-validation, while 2005 and 2006 became the training set.

In [ ]:
plt.figure(figsize=(8, 5))

plt.bar(wind_ava['year'].value_counts().index, wind_ava['year'].value_counts().values, color='skyblue', edgecolor='black')
plt.title(f'Year barplot')
plt.xlabel('Year')
plt.ylabel('Frequency')

# Muestra el histograma
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns  # Optional, but can enhance the appearance of the plot

# Assuming 'wind_ava' is your DataFrame with 'year' and 'month' columns

# Get unique years in the DataFrame
unique_years = wind_ava['year'].unique()

# Set up subplots
fig, axes = plt.subplots(nrows=len(unique_years), ncols=1, figsize=(8, 5 * len(unique_years)))

# Loop through each year and create a bar plot for the frequency of each month
for i, year in enumerate(unique_years):
    ax = axes[i]
    
    # Filter data for the current year
    data_year = wind_ava[wind_ava['year'] == year]
    
    # Create a count plot for the 'month' column
    sns.countplot(x='month', data=data_year, ax=ax, palette='viridis')
    
    # Customize the plot
    ax.set_title(f'Monthly Frequency for the Year {year}')
    ax.set_xlabel('Month')
    ax.set_ylabel('Frequency')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()


In [16]:
##chat gpt
# Splitting the data into training and test sets
train_data = wind_ava[wind_ava['year'].isin([2005, 2006, 2007])]
test_data = wind_ava[wind_ava['year'].isin([2008, 2009])]

# Further split the training set into train-train and train-validation
train_train_data = train_data[train_data['year'].isin([2005, 2006])]
train_validation_data = train_data[train_data['year'] == 2007]


# Features and target for training set
X_train_train = train_train_data.drop('energy', axis=1)
#y_train_train = train_train_data['energy']

X_train = train_data.drop('energy', axis=1)
y_train = train_data['energy']


# Features and target for validation set
X_train_validation = train_validation_data.drop('energy', axis=1)
#y_train_validation = train_validation_data['energy']

# Features and target for test set
X_test = test_data.drop('energy', axis=1)
y_test = test_data['energy']


In [ ]:
#X_train_train

In [ ]:
#y_train_train

### 2.2. Metric

CHAT GPT:
The choice of which metric to use for evaluating a regression model depends on the specific characteristics of the problem and our goals. As a summary of the benefits and drawbacks of the most commonly used regression metrics:

- Mean Squared Error (MSE) or Root Mean Squared Error (RMSE): These are often preferred when large errors should be penalized more heavily, and the distribution of errors is approximately normal. However, if your data contains outliers, MSE and RMSE can be significantly affected.

- Mean Absolute Error (MAE): These are more robust to outliers. If your dataset has a skewed distribution or contains outliers, MAE might be more appropriate.

Therefore, we will evaluate the outliers of our objetive variable to understand which metric is better on this problem. As observed, the data is rigth skewed, with many outliers on the rigth section, therefore, a MAE metric would affect less to these errors when predicting outliers.

In [ ]:
plt.boxplot(train_train_data['energy'], vert=False, widths=0.7, patch_artist=True, boxprops=dict(facecolor='skyblue', color='black'))
plt.title('Energy Boxplot')
plt.xlabel('Energy value')
plt.show()

### 2.3. Strategy

Issues to address: 
- Feature selection: are all the 550 attributes really necessary? Maybe only the attributes related to the Sotavento location (13th location in the grid) are actually required? Or only the attributes related to wind? - Imputation: Does imputation improve performance in this problem? Which method seem to work best?- Modeling:s) Which method is more appropriate to the proble). Does hyper-parameter tuning contribute to improve performance? At what cost (compute tim?). Which HPO method does perform bette

Strategy:

The idea is to cover the first two issues on the preprocessing on the data, starting first with the feature selection in order to reduce the data size as much as posible for making the later computations easier. During this preprocessing, on each stage we compute a pipeline where we perform the necesary transformations (defining the adequate search space on each stage), and later we predict with a KNN. This method was chosen due to the simplicity and the small number of hyperparamters it has (it was used on default mode, not HPO of KNN during the preprocessing). Appart from feature selection and imputation, also scaling hyperparameters will be tunned during this process. All this tuning will be donde by performing a GridSearch over the hyperparameters in order to be more exhaustive, as the process of preprocessing is the most important in the project. When preprocessing is finished, we will apply all the necessary transformations to the data for having it ready for the modeling.

Next step is to evaluate different models/methods for the problem. First, we would define a naive model with the median of the data as prediction. Secondly, we would address four different methods: Trees, KNN, gradient and boosting. All of them would be modelled both default and with HPO. Among HPO, it is difficult to try all the possible methods, so we have chosen to try Random Search, Grid Search, Bayesian Search, and Succesive Halving. All the metric results and time measurements would be kept for later comparison.

New issue: ).

## 3. Preprocessing 

### 3.1. Feature selection

Explicar aqui que no todas las features son necesarias y que las eligiremos en función de dos métricas y 5 rangos de número de features seleccionadas

In [17]:
#We set a seed for reproducibility, althougth the split is predefined, some steps in all the pipelines defined may have some randomness
random_state= 100516919

# Load a sample dataset for demonstration
X = X_train
y = y_train

# Define the holdout split
indices= ([-1] * (len(X_train_train))) + ([0] * (len(X_train_validation)))
inner= PredefinedSplit(indices)

pipeline_feature_selection = Pipeline([
    ('imputer', SimpleImputer()), 
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest()),  # Placeholder, will be replaced during grid search
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

# Define the hyperparameter grid for GridSearchCV
param_grid_fs = {
    'feature_selection__score_func': [f_regression, mutual_info_regression],
    'feature_selection__k': [100,150,250,300,350]  # Adjust as needed
}


# Use RandomizedSearchCV to find the best combination of parameters
grid_search = RandomizedSearchCV(pipeline_feature_selection, param_grid_fs, cv=inner, scoring='neg_mean_absolute_error', n_jobs=-1, random_state= random_state )
grid_search.fit(X, y)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Best Model MAE: {mae}")

Best Model MAE: 314.9798252957234


In [18]:
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

Best parameters: {'feature_selection__score_func': <function mutual_info_regression at 0x13f757310>, 'feature_selection__k': 250}


In [19]:
# Access the selected features
selected_features = best_model.named_steps['feature_selection'].get_support()

# Print the indices of selected features
selected_feature_indices = np.where(selected_features)[0]
print(f"Indices of selected features: {selected_feature_indices}")

Indices of selected features: [ 79  80  81  82  83  84  85  87  89  90  91  93  94  96 102 103 154 155
 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191
 192 193 194 195 196 197 198 200 201 202 203 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278
 279 280 281 282 283 284 285 286 287 288 289 290 291 292 294 295 296 297
 298 299 300 301 302 303 379 380 381 382 383 384 385 386 387 388 389 390
 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408
 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426
 427 428 479 480 481 482 484 485 486 487 488 489 490 504 505 506 507 508
 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526
 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544
 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562
 563 564 565 566 567 

In [20]:
column_indices = X.columns
numerical_indices = range(len(X.columns))
for numerical_index, column_label in enumerate(X.columns):
    print(f"Column Label: {column_label}, Numerical Index: {numerical_index}")


Column Label: year, Numerical Index: 0
Column Label: month, Numerical Index: 1
Column Label: day, Numerical Index: 2
Column Label: hour, Numerical Index: 3
Column Label: p54.162.1, Numerical Index: 4
Column Label: p54.162.2, Numerical Index: 5
Column Label: p54.162.3, Numerical Index: 6
Column Label: p54.162.4, Numerical Index: 7
Column Label: p54.162.5, Numerical Index: 8
Column Label: p54.162.6, Numerical Index: 9
Column Label: p54.162.7, Numerical Index: 10
Column Label: p54.162.8, Numerical Index: 11
Column Label: p54.162.9, Numerical Index: 12
Column Label: p54.162.10, Numerical Index: 13
Column Label: p54.162.11, Numerical Index: 14
Column Label: p54.162.12, Numerical Index: 15
Column Label: p54.162.13, Numerical Index: 16
Column Label: p54.162.14, Numerical Index: 17
Column Label: p54.162.15, Numerical Index: 18
Column Label: p54.162.16, Numerical Index: 19
Column Label: p54.162.17, Numerical Index: 20
Column Label: p54.162.18, Numerical Index: 21
Column Label: p54.162.19, Numer

explicar aqui que filtramos el dataset original con las features que nos ha dado la pipeline. Destacas que las seleccionadas son: 
- p59.162: Vertical integral of divergence of kinetic energy
- u10n: Neutral wind at 10 m u-component
- v10n: Neutral wind at 10 m v-component
- u10: 10 metre U wind component
-• v10: 10 metre V wind componen
- iews: Instantaneous eastward turbulent surface stress
- inss: Instantaneous northward turbulent surface
- flsr: Forecast logarithm of surface roughness for heat
- u100: 100 metre U wind component
-  v100: 100 metre V wind componen
- wind_speed:  modulus of u100 and v100tt

In [21]:
# Keep only the selected features in your original data X
X_selected_features = X.iloc[:, selected_feature_indices]
X_test = X_test.iloc[:, selected_feature_indices]
X_selected_features.columns


Index(['p59.162.1', 'p59.162.2', 'p59.162.3', 'p59.162.4', 'p59.162.5',
       'p59.162.6', 'p59.162.7', 'p59.162.9', 'p59.162.11', 'p59.162.12',
       ...
       'wind_speed16', 'wind_speed17', 'wind_speed18', 'wind_speed19',
       'wind_speed20', 'wind_speed21', 'wind_speed22', 'wind_speed23',
       'wind_speed24', 'wind_speed25'],
      dtype='object', length=250)

### 3.2. Imputation


Does imputation improve performance in this problem? Which method seem to work best?

Intentamos primero elegir tipos de simple imputer, y luego ver si el iterative baja el MAE más.

In [30]:
# Load a sample dataset for demonstration
X = X_selected_features
y = y_train

# Define the holdout split
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Create a new pipeline with imputation after feature selection
pipeline_simple_imputer = Pipeline([
    ('imputer', SimpleImputer()), 
    ('scaler', StandardScaler()),  # Placeholder, will be replaced during grid search
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

# Define the hyperparameter grid for GridSearchCV
param_grid_si = {
    'imputer__strategy': ['mean', 'median', 'most_frequent']
}

# Use GridSearchCV to find the best combination of parameters
grid_search = RandomizedSearchCV(pipeline_simple_imputer, param_grid_si, cv=inner, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=random_state)
grid_search.fit(X, y)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Best Model MAE: {mae}")
# Print the selected imputation strategy
selected_imputer_strategy = best_model.named_steps['imputer'].strategy
print(f"Selected Imputation Strategy: {selected_imputer_strategy}")

Best Model MAE: 320.0383530482257
Selected Imputation Strategy: median


In [ ]:
# Load a sample dataset for demonstration
X = X_selected_features
y = y_train

# Define the holdout split
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Create a new pipeline with imputation after feature selection
pipeline_iterative_imputer = Pipeline([
    ('imputer', IterativeImputer()), 
    ('scaler', StandardScaler()),  # Placeholder, will be replaced during grid search
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

pipeline_iterative_imputer.fit(X, y)

# Make predictions on the test set
y_pred = pipeline_iterative_imputer.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Iterative imputer MAE: {mae}")


In [ ]:
# Load a sample dataset for demonstration
X = X_selected_features
y = y_train

# Define the holdout split
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Create a new pipeline with imputation after feature selection
pipeline_KNN_imputer = Pipeline([
    ('imputer', KNNImputer()), 
    ('scaler', StandardScaler()),  
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

pipeline_KNN_imputer.fit(X, y)

# Make predictions on the test set
y_pred = pipeline_KNN_imputer.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"KNN imputer MAE: {mae}")

Como el MAE es menor para el iterative, usamos este para transformar nuestros datos y continuar al último paso del preprocesado, el scaling

In [ ]:
# Access the imputed data
X_imputed = pipeline_iterative_imputer.named_steps['imputer'].transform(X)
X_imputed 
X_test = pipeline_iterative_imputer.named_steps['imputer'].transform(X_test)
X_test

### 3.3. Scaling

Por último, ahora que ya tenemos las mejores features seleccionadas y habiendo imputado con el iterative, hay que ver qué método de escalado es mejor (esto luego solo afecta a los métodos que calculan distancias como el knn pero se aplicará al dataset para todos los casos)


In [ ]:
X = X_imputed
y = y_train

# Define the holdout split
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Create a new pipeline with scaling after feature selection and imputation
pipeline_standard_scaler = Pipeline([
    ('scaler', StandardScaler()), 
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

pipeline_standard_scaler.fit(X, y)

# Make predictions on the test set
y_pred = pipeline_standard_scaler.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Standard scaler MAE: {mae}")

In [ ]:
# Create a new pipeline with scaling after feature selection and imputation
pipeline_minmax_scaler = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

pipeline_minmax_scaler.fit(X, y)

# Make predictions on the test set
y_pred = pipeline_minmax_scaler.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"MinMax scaler MAE: {mae}")

In [ ]:
# Create a new pipeline with scaling after feature selection and imputation
pipeline_robust_scaler = Pipeline([
    ('scaler', RobustScaler()), 
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

pipeline_robust_scaler.fit(X, y)

# Make predictions on the test set
y_pred = pipeline_robust_scaler.predict(X_test)

# Calculate and print the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Robust scaler MAE: {mae}")

### 3.4. Final dataset

In [ ]:
X_train = pipeline_robust_scaler.named_steps['scaler'].transform(X)
X_train
X_test = pipeline_robust_scaler.named_steps['scaler'].transform(X_test)
X_test

In [18]:
# Assuming X_train and X_test are NumPy arrays
#np.savetxt('X_train.csv', X_train, delimiter=';')
#np.savetxt('X_test.csv', X_test, delimiter=';')
#np.savetxt('X_train_train.csv', X_train_train, delimiter=';')
#np.savetxt('X_train_validation.csv', X_train_validation, delimiter=';')

In [5]:
X_train = np.loadtxt('X_train.csv', delimiter=';')
X_test = np.loadtxt('X_test.csv', delimiter=';')
X_train_train = np.loadtxt('X_train_train.csv', delimiter=';')
X_train_validation = np.loadtxt('X_train_validation.csv', delimiter=';')
X_test

array([[-0.31177367, -0.31696642, -0.31655972, ..., -0.23377745,
        -0.21241713, -0.19716302],
       [-0.09378879, -0.10216953, -0.10638298, ...,  0.11043825,
         0.10968315,  0.1042859 ],
       [ 0.05649893,  0.04862603,  0.04390409, ...,  0.70883047,
         0.67642773,  0.64116408],
       ...,
       [ 0.54200045,  0.58215538,  0.62177319, ..., -0.2946182 ,
        -0.25202736, -0.21299333],
       [ 0.09334912,  0.09679973,  0.10300574, ...,  0.88391007,
         0.86213978,  0.84281453],
       [ 0.06947422,  0.07564338,  0.08476866, ...,  0.47437584,
         0.52065433,  0.57001822]])

## 4. Modeling


### 4.1. Dummy model

In [ ]:
# Create a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'MAE', 'Training Time'])

# Function to add results to the DataFrame
def add_result(model_name, mae, training_time):
    results_df.loc[len(results_df)] = [model_name, mae, training_time]

indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

dummy_reg =DummyRegressor(strategy='median')

# Measure training time
start_time = time.time()

# Fit the model and calculate scores
dummy_scores = cross_val_score(dummy_reg, X_train, y_train, cv=inner, scoring='neg_mean_absolute_error')

# Measure end time
end_time = time.time()

training_time= end_time-start_time

# Add results to the DataFrame
add_result('Dummy Mean', -dummy_scores.mean(), training_time)

# Print the results DataFrame
results_df
        

### 4.2. Trees

#### 4.2.1 Default hyperparameters

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=random_state)

# Measure training time
start_time = time.time()

tree_default_scores = cross_val_score(tree_reg,  X_train, y_train, cv=inner, scoring='neg_mean_absolute_error')

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

add_result('Tree Default', -tree_default_scores.mean(), training_time)

results_df

#### 4.2.2. HPO

Decision trees hyperparameters:
- max_depth: the maximum depth of the tree
- 	min_samples_split: the minimum number of instances to splits a node (the default value is 2: if a node contains fewer  than instances, the node is not split)


##### Random Search

In [ ]:
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

param_random_tree = {'max_depth': [2,4,6,8,10,12,14,16,18,20],
                     'min_samples_split': [100,110,120,130]}

random_search_tree = RandomizedSearchCV(tree_reg, param_random_tree, cv= inner, scoring='neg_mean_absolute_error', random_state= random_state) 

# Measure training time
start_time = time.time()

random_search_tree.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

#We print the selected parameters to be sure we are not on the boundaries of the search space

random_search_tree.best_params_


In [ ]:
add_result('Tree Random HPO', -random_search_tree.best_score_, training_time)

results_df

##### Grid Search

In [ ]:
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

param_grid_tree = {'max_depth': [2,4,6,8,10,12,14,16,18,20],
                     'min_samples_split': [100,110,120,130]}

grid_search_tree = GridSearchCV(tree_reg, param_grid_tree, cv= inner, scoring='neg_mean_absolute_error') 

# Measure training time
start_time = time.time()

grid_search_tree.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

add_result('Tree Grid HPO', -grid_search_tree.best_score_, training_time)

results_df

##### Bayes optimization

In [ ]:
from skopt.space import Integer
from skopt import BayesSearchCV

indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

param_bayes_tree = {'max_depth': Integer(2, 20),
                     'min_samples_split': Integer(100, 130)}

budget=30 #number of iterations
bayes_search_tree = BayesSearchCV(tree_reg, 
                                 param_bayes_tree, 
                                 cv= inner, 
                                 scoring='neg_mean_absolute_error',
                                n_iter=budget,
                                n_jobs=-1, verbose=1) 


# Measure training time
start_time = time.time()

bayes_search_tree.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

add_result('Tree Bayes HPO', -bayes_search_tree.best_score_, training_time)



In [ ]:
results_df

##### Halving Grid Search

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

param_halving_tree = {'max_depth': list(range(2, 20, 1)),
                     'min_samples_split':list(range(100, 130, 1))}

halving_search_tree = HalvingGridSearchCV(tree_reg, 
                                 param_halving_tree, 
                                 cv= inner, 
                                 scoring='neg_mean_absolute_error',
                                factor= 2, #proportion of candidates that are selected in each iteration
                                min_resources='exhaust', #“exhaust” means that first iteration is as large as possible, so that in the last iteration, the entire training data is used.
                                max_resources='auto',
                                n_jobs=-1, verbose=1) 


# Measure training time
start_time = time.time()

halving_search_tree.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

add_result('Tree Halving HPO', -halving_search_tree.best_score_, training_time)

In [ ]:
results_df

### 4.3. KNN

#### 4.3.1. Default hyperparameters

In [ ]:
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Create a KNN regressor with default parameters
knn_reg = KNeighborsRegressor(n_jobs=-1)

# Measure training time
start_time = time.time()

# Perform cross-validation and calculate mean absolute error
knn_scores = cross_val_score(knn_reg, X_train, y_train, cv=inner, scoring='neg_mean_absolute_error')

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Add the results to a dataframe or any data structure you're using to store results
add_result('KNN Default', -knn_scores.mean(), training_time)

# Assuming 'add_result' is a function that adds results to your 'results_df'
results_df

#### 4.3.2. HPO

##### Random Search

In [ ]:
# Define the parameter grid for random search
param_dist = {
    'n_neighbors': [3, 5, 7, 9, 11],  
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Create a KNN regressor with default parameters
knn_reg = KNeighborsRegressor()

# Define the randomized search object
random_search = RandomizedSearchCV(knn_reg, param_distributions=param_dist, scoring='neg_mean_absolute_error', cv=inner, n_jobs=-1,random_state= random_state)

# Measure training time
start_time = time.time()

# Fit the randomized search to the data
random_search.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Print the mean absolute error on the validation set
print("Validation MAE:", -random_search.best_score_)

# Print training time
print("Training Time:", training_time)

In [ ]:
add_result('KNN Random HPO', -random_search.best_score_, training_time)

results_df

##### Grid Search

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Create a KNN regressor with default parameters
knn_reg = KNeighborsRegressor()

# Define the grid search object
grid_search_knn = GridSearchCV(knn_reg, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=inner, n_jobs=-1)

# Measure training time
start_time = time.time()

# Fit the grid search to the data
grid_search_knn.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search_knn.best_params_)

# Print the mean absolute error on the validation set
print("Validation MAE:", -grid_search_knn.best_score_)

In [ ]:
# Add the results to the dataframe
add_result('KNN Grid HPO', -grid_search_knn.best_score_, training_time)

# Display the results dataframe
results_df

##### Bayes optimization

In [ ]:
# Define the parameter search space for Bayesian optimization
param_space = {
    'n_neighbors': [3, 5, 7, 9, 11], 
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Create a KNN regressor with default parameters
knn_reg = KNeighborsRegressor()

budget=20
# Define the Bayesian search object
bayes_search_knn = BayesSearchCV(knn_reg, search_spaces=param_space, scoring='neg_mean_absolute_error', cv=inner, n_jobs=-1, n_iter=budget, verbose=1)

# Measure training time
start_time = time.time()

# Fit the Bayesian search to the data
bayes_search_knn.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Print the best hyperparameters
print("Best Hyperparameters:", bayes_search_knn.best_params_)

# Print the mean absolute error on the validation set
print("Validation MAE:", -bayes_search_knn.best_score_)

In [ ]:
# Add the results to the dataframe
add_result('KNN Bayes HPO', -bayes_search_knn.best_score_, training_time)

# Display the results dataframe
results_df

##### Halving Grid Search

In [ ]:
# Create a KNeighborsRegressor instance
knn_reg = KNeighborsRegressor()

# Create a PredefinedSplit to split the data into training and validation sets
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Define the parameter search space for the Halving Grid Search
param_halving_knn = {
    'n_neighbors': list(range(3, 12, 2)),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Create a HalvingGridSearchCV instance for KNN
halving_search_knn = HalvingGridSearchCV(
    knn_reg,
    param_halving_knn,
    cv=inner,
    scoring='neg_mean_absolute_error',
    factor=2,  # proportion of candidates that are selected in each iteration
    min_resources='exhaust',  # “exhaust” means that the first iteration is as large as possible
    max_resources='auto',
    n_jobs=-1,
    verbose=1
)

# Measure training time
start_time = time.time()

halving_search_knn.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Print the best parameters and best score
print("Best Parameters:", halving_search_knn.best_params_)
print("Best Score:", -halving_search_knn.best_score_)
print("Training Time:", training_time)

In [ ]:
add_result('KNN Halving HPO', -halving_search_knn.best_score_, training_time)

results_df

### 4.4. Ensemble 1: Random Forest

#### 4.4.1. Default hyperparameters

In [ ]:
rand_for_reg = RandomForestRegressor(random_state=random_state)

# Measure training time
start_time = time.time()

rand_for_default_scores = cross_val_score(rand_for_reg,  X_train, y_train, cv=inner, scoring='neg_mean_absolute_error')

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

add_result('RF Default', -rand_for_default_scores.mean(), training_time)

results_df



#### 4.4.2. HPO

Random forest Hyperparameters:
- n_estimators: number of trees (base models) in the
ensemble: there should be enough trees in the
ensemble. The default (100) usually works well, but it
can also be tuned.
- max_features : mtry or m. Size of the random subset
of features to be compared at every node. Possible
values: sqrt, log2, or a number between 1 and the
maximum number of features in the dataset. It can also be a real number between 0 and 1 (fraction of
features)
- max_depth / min_samples_split: hyper parameters of
the trees (base models
- njobs is not a hyper parameter, but allows the RF to
be trained in parallel

##### Random Search

In [ ]:
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

param_random_RF = {'max_depth': [2,4,6,8,10,12,14,16,18,20],
                     'min_samples_split': [100,110,120,130],
                  'n_estimators':[25,50,100,150],
                  'max_features':[0.2,0.4,0.6,0.8,1]}

random_search_RF = RandomizedSearchCV(rand_for_reg, param_random_RF, cv= inner, scoring='neg_mean_absolute_error', random_state= random_state) 

# Measure training time
start_time = time.time()

random_search_RF.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

#We print the selected parameters to be sure we are not on the boundaries of the search space

random_search_RF.best_params_

In [ ]:
add_result('RF Random HPO', -random_search_RF.best_score_, training_time)

In [ ]:
results_df

##### Grid Search and Bayesian Search

As we have already observed with the HPO of the trees, Grid Search and Bayesian Search takes too much time compared to the minimization of the MAE. Therefore, as RF is a ensemble model based on trees, we may expect the same behaviour, but even with a higher increase (as already the random search has increased the computing time a lot). Therefore, we would not try to do HPO neither with Grid or Bayesian Search.

##### Halving Grid Search

In [ ]:
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

param_halving_RF = {'max_depth': list(range(2, 20, 2)),
                     'min_samples_split':list(range(100, 130, 5)),
                     'n_estimators':list(range(50,150, 50)),
                       'max_features':[0.2,0.4,0.6,0.8,1]}


halving_search_RF = HalvingGridSearchCV(rand_for_reg, 
                                 param_halving_RF, 
                                 cv= inner, 
                                 scoring='neg_mean_absolute_error',
                                factor= 2, #proportion of candidates that are selected in each iteration
                                min_resources='exhaust', #“exhaust” means that first iteration is as large as possible, so that in the last iteration, the entire training data is used.
                                max_resources='auto',
                                n_jobs=-1, verbose=1) 


# Measure training time
start_time = time.time()

halving_search_RF.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time


halving_search_RF.best_params_



In [ ]:
add_result('RF Halving HPO', -halving_search_RF.best_score_, training_time)
results_df

In [ ]:
results_df

### 4.5. Ensemble 2: Gradient Boosting
custom loss function for boosting!!!

#### 4.5.1. Default hyperparameters

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Create a GradientBoostingRegressor instance with default parameters
gb_reg = GradientBoostingRegressor(random_state=random_state)

# Measure training time
start_time = time.time()

# Use cross_val_score for cross-validation
gb_default_scores = cross_val_score(gb_reg, X_train, y_train, cv=inner, scoring='neg_mean_absolute_error')

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

In [ ]:
# Add the results to your results dataframe or store them as needed
add_result('GB Default', -gb_default_scores.mean(), training_time)

# Display or save your results dataframe
results_df

#### 4.5.2. HPO

##### Random Search

In [ ]:
# Create a GradientBoostingRegressor instance
gb_reg = GradientBoostingRegressor()

# Create a PredefinedSplit to split the data into training and validation sets
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Define the parameter search space for RandomizedSearchCV
param_random_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
}

# Create a RandomizedSearchCV instance for Gradient Boosting
random_search_gb = RandomizedSearchCV(
    gb_reg,
    param_distributions=param_random_gb,
    cv=inner,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1,
    random_state=random_state  # Set a random seed for reproducibility
)

# Measure training time
start_time = time.time()

random_search_gb.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Print the best parameters and best score
print("Best Parameters:", random_search_gb.best_params_)
print("Best Score:", -random_search_gb.best_score_)
print("Training Time:", training_time)

In [ ]:
# Add the results to your results dataframe or store them as needed
add_result('GB Random HPO', -random_search_gb.best_score_, training_time)

# Display or save your results dataframe
results_df

##### Halving Grid Search

In [6]:
# Create a GradientBoostingRegressor instance
gb_reg = GradientBoostingRegressor()

# Create a PredefinedSplit to split the data into training and validation sets
indices = ([-1] * len(X_train_train)) + ([0] * len(X_train_validation))
inner = PredefinedSplit(indices)

# Define the parameter search space for the Halving Grid Search
param_halving_gb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10]
}

# Create a HalvingGridSearchCV instance for Gradient Boosting
halving_search_gb = HalvingGridSearchCV(
    gb_reg,
    param_halving_gb,
    cv=inner,
    scoring='neg_mean_absolute_error',
    factor=2,  # proportion of candidates that are selected in each iteration
    min_resources='exhaust',  # “exhaust” means that the first iteration is as large as possible
    max_resources='auto',
    n_jobs=-1,
    verbose=1
)

# Measure training time
start_time = time.time()

halving_search_gb.fit(X_train, y_train)

# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time

# Print the best parameters and best score
print("Best Parameters:", halving_search_gb.best_params_)
print("Best Score:", -halving_search_gb.best_score_)
print("Training Time:", training_time)


n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 57
max_resources_: 3649
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 108
n_resources: 57
Fitting 1 folds for each of 108 candidates, totalling 108 fits
----------
iter: 1
n_candidates: 54
n_resources: 114
Fitting 1 folds for each of 54 candidates, totalling 54 fits
----------
iter: 2
n_candidates: 27
n_resources: 228
Fitting 1 folds for each of 27 candidates, totalling 27 fits
----------
iter: 3
n_candidates: 14
n_resources: 456
Fitting 1 folds for each of 14 candidates, totalling 14 fits
----------
iter: 4
n_candidates: 7
n_resources: 912
Fitting 1 folds for each of 7 candidates, totalling 7 fits
----------
iter: 5
n_candidates: 4
n_resources: 1824
Fitting 1 folds for each of 4 candidates, totalling 4 fits
----------
iter: 6
n_candidates: 2
n_resources: 3648
Fitting 1 folds for each of 2 candidates, totalling 2 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_

In [7]:
from joblib import dump, load
# Save the model to a file
joblib.dump(halving_search_gb, 'halving_search_gb_model.joblib')

# Load the model from the file
halving_search_gb = joblib.load('halving_search_gb_model.joblib')


NameError: name 'joblib' is not defined

In [ ]:
# Add the results to your results dataframe or store them as needed
add_result('GB Halving HPO', -halving_search_gb.best_score_, training_time)

# Display or save your results dataframe
results_df

In [ ]:
min_mae_index = results_df['MAE'].idxmin()
results_df.loc[min_mae_index]

### 4.6. Optuna

In [ ]:
import optuna
def objective(trial):
    method = trial.suggest_categorical('method', ['tree', 'knn', 'random_forest', 'gradient_boosting'])
    if method== 'tree':
        max_depth=trial.suggest_int('max_depth', 2, 20)
        min_samples_split= trial.suggest_int('min_samples_split', 50, 150, step=10)
        params = {'max_depth': max_depth, 'min_samples_split': min_samples_split}
        regr = DecisionTreeRegressor(random_state=random_state, **params)
        
    elif method=='knn':
        n_neighbors= trial.suggest_int('n_neighbors',1,50)
        weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
        algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
        metric= trial.suggest_categorical('metric', ['minkowski','euclidean', 'chebyshev', 'manhattan'])

        if metric == 'minkowski':
            p= trial.suggest_float('p', 1.0,3.0)
            params = {'n_neighbors': n_neighbors,'weights': weights,'algorithm': algorithm, 'metric': metric, 'p': p }
        else:
            params = {'n_neighbors': n_neighbors,'weights': weights,'algorithm': algorithm, 'metric': metric}
        
        regr = KNeighborsRegressor(**params)

    elif method=='random_forest':
        max_depth=trial.suggest_int('max_depth', 2, 20)
        min_samples_split= trial.suggest_int('min_samples_split', 50, 150,step= 10)
        n_estimators= trial.suggest_int('n_estimators', 50, 150,step=10)
        max_features= trial.suggest_float('max_features', 0.2, 1.0)
        params = {'max_depth': max_depth, 'min_samples_split': min_samples_split,'n_estimators': n_estimators, 'max_features': max_features }
        regr = RandomForestRegressor(random_state=random_state, **params)

    elif method=='gradient_boosting':
        max_depth=trial.suggest_int('max_depth', 2, 20)
        min_samples_split= trial.suggest_int('min_samples_split', 50, 150, step=10)
        n_estimators= trial.suggest_int('n_estimators', 50, 150, step=10)
        learning_rate= trial.suggest_float('learning_rate', 0.01, 0.21, step=0.02)
        params = {'max_depth': max_depth, 'min_samples_split': min_samples_split,'n_estimators': n_estimators, 'learning_rate': learning_rate }
        regr = GradientBoostingRegressor(random_state=random_state, **params)

    scores= cross_val_score(regr, X_train, y_train, scoring= 'neg_mean_absolute_error',
                            n_jobs=-1, cv=inner)
    return scores.mean()

budget=15
sampler= optuna.samplers.TPESampler(seed=random_state)
study = optuna.create_study(direction='maximize', sampler=sampler)
start_time = time.time()

study.optimize(objective,n_trials=budget)


In [ ]:
# Measure end time
end_time = time.time()

# Calculate training time
training_time = end_time - start_time
print(study.best_params)
print(-study.best_value)
print(training_time)

## 5. Best model 

### 5.1. Estimation of the accuracy

Using the best alternative (based on inner evaluation), make an estimation of the accuracy that the final model might get on future data (outer evaluation).

In [20]:
predictions=halving_search_gb.predict(X_test)

In [21]:
mae = mean_absolute_error(y_test, predictions)

print(f"MAE: {mae}")


MAE: 282.4050298313833


### 5.2. Training final model and predict

Train (using ALL THE DATA) and use it to make predictions on the “competition data”. Save both the final model and the competition predictions on files.

In [4]:
X = wind_ava.drop('energy', axis=1)
y=wind_ava['energy']

In [6]:
# Define the parameters
gb_params = {
    'learning_rate': 0.1,
    'max_depth': 3,
    'min_samples_split': 5,
    'n_estimators': 100
}

# Create the pipeline
pipeline_final = Pipeline([
    ('imputer', IterativeImputer()), 
    ('scaler', RobustScaler()),
    ('feature_selection', SelectKBest(k=150, score_func=mutual_info_regression)),  
    ('gb', GradientBoostingRegressor(**gb_params))
])

In [7]:
pipeline_final.fit(X,y)

KeyboardInterrupt: 

In [8]:
wind_competition = pd.read_csv("/Users/pameladiaz/Desktop/GitHub/Big_Data_Intelligence/wind_competition.csv.gzip", compression="gzip")

In [9]:
pd.set_option('display.max_rows', 10)
wind_competition

,year,month,day,hour,p54.162.1,p54.162.2,p54.162.3,p54.162.4,p54.162.5,p54.162.6,...,v100.16,v100.17,v100.18,v100.19,v100.20,v100.21,v100.22,v100.23,v100.24,v100.25
0,2010,1,1,0,2.403131e+06,2.395445e+06,2.387755e+06,2.380065e+06,2.372380e+06,2.399548e+06,...,7.212586,7.057422,6.901760,NaN,6.591434,7.184147,7.030980,6.877313,6.723647,6.570479
1,2010,1,1,6,2.410306e+06,2.402394e+06,NaN,2.386571e+06,2.378660e+06,NaN,...,0.207289,0.583972,NaN,NaN,1.714518,0.345988,0.723170,1.100850,1.478031,1.855712
2,2010,1,1,12,2.434908e+06,2.426793e+06,2.418683e+06,2.410573e+06,2.402462e+06,2.431465e+06,...,1.670114,1.691568,1.712522,1.733976,NaN,1.664127,1.682587,1.700548,NaN,1.736470
3,2010,1,1,18,2.447112e+06,NaN,2.431027e+06,2.422984e+06,2.414942e+06,2.443696e+06,...,1.217597,1.278464,1.339332,1.399701,1.460569,1.215102,1.272477,1.329853,NaN,1.444105
4,2010,1,2,0,2.459695e+06,NaN,2.443809e+06,2.435866e+06,NaN,2.456252e+06,...,3.755089,3.686738,NaN,3.549536,3.481184,3.781532,3.710686,NaN,3.569492,3.498646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2010,12,30,18,2.450279e+06,2.442801e+06,2.435324e+06,2.427846e+06,2.420368e+06,2.446240e+06,...,3.473201,3.445761,NaN,3.391379,3.363938,3.499644,3.467214,NaN,3.401856,3.369426
1185,2010,12,31,0,2.455407e+06,2.447817e+06,2.440226e+06,2.432635e+06,2.425045e+06,2.451400e+06,...,2.280789,2.372091,2.463892,2.555194,2.646994,2.333674,2.418490,2.503306,2.588621,2.673437
1186,2010,12,31,6,2.457296e+06,2.449624e+06,2.441947e+06,2.434271e+06,2.426599e+06,2.453288e+06,...,2.211939,2.341657,2.470877,NaN,2.729815,2.188489,2.312221,2.436451,2.560183,2.684413
1187,2010,12,31,12,2.464015e+06,2.456257e+06,2.448494e+06,NaN,2.432974e+06,2.459993e+06,...,NaN,1.311393,1.387228,1.463563,1.539897,1.263996,1.338335,1.412174,NaN,1.560852


### 5.3. Model interpretation

In [8]:
import shap

# Assuming model is your trained gradient boosting model
explainer = shap.TreeExplainer(halving_search_gb)
shap_values = explainer.shap_values(X_train)  # X is your input data

# Plot summary plot
shap.summary_plot(shap_values, X_train)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


InvalidModelError: Model type not yet supported by TreeExplainer: <class 'sklearn.model_selection._search_successive_halving.HalvingGridSearchCV'>

In [22]:
# Select only the headers from the original DataFrame
selected_feature_headers = X_selected_features.columns

# Create a new DataFrame using the selected headers and columns of X_train
X_prueba = pd.DataFrame(X_train, columns=selected_feature_headers)
X_prueba

,p59.162.1,p59.162.2,p59.162.3,p59.162.4,p59.162.5,p59.162.6,p59.162.7,p59.162.9,p59.162.11,p59.162.12,...,wind_speed16,wind_speed17,wind_speed18,wind_speed19,wind_speed20,wind_speed21,wind_speed22,wind_speed23,wind_speed24,wind_speed25
0,1.400898e+06,1.405015e+06,1.408953e+06,1.413070e+06,1.417008e+06,1.389443e+06,1.393023e+06,1.400362e+06,1.380314e+06,1.383536e+06,...,5.134636,NaN,NaN,NaN,4.393808,5.137268,4.948249,4.760744,4.574870,4.389944
1,1.167130e+06,1.171426e+06,1.175722e+06,1.180018e+06,1.184314e+06,1.155674e+06,1.159433e+06,1.167130e+06,1.146367e+06,1.149946e+06,...,5.298553,NaN,4.869726,4.655077,4.440835,5.300003,5.085998,4.872320,4.658335,NaN
2,NaN,1.114147e+06,NaN,1.115758e+06,1.116653e+06,1.104661e+06,1.105377e+06,1.106450e+06,1.098038e+06,1.098396e+06,...,5.587899,NaN,NaN,NaN,NaN,5.529130,5.324473,5.122190,4.922326,NaN
3,1.090699e+06,1.090878e+06,1.091057e+06,1.091236e+06,1.091594e+06,1.082644e+06,1.082644e+06,1.082823e+06,1.076200e+06,1.076200e+06,...,4.779456,4.715311,4.655777,4.600019,4.549180,4.698198,4.634036,4.573135,4.516475,4.463542
4,1.093563e+06,NaN,NaN,1.081749e+06,1.077811e+06,1.085329e+06,NaN,NaN,1.078885e+06,1.074947e+06,...,3.941046,NaN,NaN,3.917355,NaN,NaN,3.851946,3.843097,3.836565,3.832704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,8.506655e+05,8.427897e+05,NaN,8.272171e+05,8.193413e+05,NaN,8.171933e+05,8.023367e+05,8.043056e+05,7.969668e+05,...,NaN,7.471601,7.279755,NaN,6.895840,7.371665,NaN,6.981368,6.786564,6.591311
3645,1.327868e+06,1.311759e+06,1.295828e+06,1.279718e+06,1.263609e+06,1.294575e+06,1.278644e+06,1.246783e+06,1.268263e+06,1.252511e+06,...,NaN,6.912081,NaN,6.633936,6.496771,6.868302,NaN,6.577254,6.433403,6.290071
3646,1.630013e+06,1.622495e+06,NaN,1.607460e+06,1.599942e+06,1.618199e+06,1.609966e+06,1.593319e+06,1.608713e+06,1.599942e+06,...,5.251092,5.224762,5.200617,5.177566,5.155690,5.070743,5.042703,5.016349,4.991320,NaN
3647,4.058611e+05,4.051451e+05,4.042501e+05,4.035342e+05,4.028182e+05,NaN,4.004912e+05,3.990593e+05,3.976273e+05,3.969113e+05,...,NaN,2.655181,NaN,NaN,2.487387,2.663482,2.607470,NaN,2.495662,2.439739


In [24]:
# Create a new DataFrame using the selected headers and columns of X_train
X_prueba_test = pd.DataFrame(X_test, columns=selected_feature_headers)
X_prueba_test

,p59.162.1,p59.162.2,p59.162.3,p59.162.4,p59.162.5,p59.162.6,p59.162.7,p59.162.9,p59.162.11,p59.162.12,...,wind_speed16,wind_speed17,wind_speed18,wind_speed19,wind_speed20,wind_speed21,wind_speed22,wind_speed23,wind_speed24,wind_speed25
3649,7.187457e+05,NaN,6.911803e+05,6.775767e+05,6.637940e+05,NaN,7.006671e+05,6.734598e+05,NaN,6.974452e+05,...,4.860973,4.804251,NaN,4.691537,4.635573,4.805741,4.750617,4.695948,4.641535,NaN
3650,1.064565e+06,NaN,1.025365e+06,1.005676e+06,9.861653e+05,1.060807e+06,1.041117e+06,1.002096e+06,1.057764e+06,1.038253e+06,...,6.398697,NaN,NaN,5.839055,5.653117,6.311038,6.128136,5.945880,5.763317,5.580927
3651,1.302988e+06,1.283656e+06,1.264325e+06,1.244993e+06,1.225662e+06,NaN,1.281687e+06,1.242666e+06,1.299766e+06,1.280076e+06,...,9.014981,8.627820,NaN,7.854394,7.468168,8.891314,8.505001,8.118789,NaN,NaN
3652,1.979592e+06,1.914616e+06,1.849641e+06,1.784665e+06,1.719690e+06,1.985499e+06,1.920165e+06,1.789319e+06,1.990331e+06,1.924461e+06,...,11.676951,NaN,10.735430,NaN,9.805992,11.542441,NaN,10.604058,NaN,9.676562
3653,2.315567e+06,2.285137e+06,2.254708e+06,2.224458e+06,2.194029e+06,2.323264e+06,2.292834e+06,2.231976e+06,2.329528e+06,2.298920e+06,...,9.273835,9.063486,8.862578,8.671693,8.492027,9.168775,8.959526,8.759649,NaN,8.390777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4743,3.611317e+06,3.699920e+06,3.788523e+06,3.877126e+06,3.965729e+06,3.674324e+06,3.764179e+06,3.943533e+06,3.724442e+06,3.815014e+06,...,11.698059,11.580590,11.463558,11.346166,11.228817,11.704657,11.583126,NaN,11.340611,NaN
4744,2.514968e+06,2.593368e+06,NaN,NaN,2.828390e+06,2.554705e+06,2.633821e+06,2.792053e+06,2.586029e+06,2.665861e+06,...,10.444271,NaN,10.565530,NaN,10.696277,10.478256,NaN,10.580514,10.635079,NaN
4745,2.073207e+06,2.128158e+06,NaN,2.238062e+06,2.293192e+06,2.110975e+06,2.166821e+06,NaN,NaN,2.197430e+06,...,4.515215,4.541995,4.569241,4.596944,4.625096,4.418757,4.446906,4.475020,4.503584,NaN
4746,NaN,1.359908e+06,1.358297e+06,1.356687e+06,1.355076e+06,1.371543e+06,1.370111e+06,1.367247e+06,1.379598e+06,1.378345e+06,...,9.462473,NaN,NaN,NaN,7.976908,9.502296,9.123884,NaN,8.383905,8.024426


In [25]:
from lime import lime_tabular

# Create a LIME explainer
explainer = lime_tabular.LimeTabularExplainer(X_prueba.values, feature_names=X_prueba.columns)

# Explain a prediction for a specific instance
exp = explainer.explain_instance(X_test.iloc[0], halving_search_gb.predict, num_features=len(X_train.columns))

# Visualize the explanation
exp.show_in_notebook(show_table=True, show_all=False)

ValueError: Domain error in arguments. The `scale` parameter must be positive for all distributions, and many distributions have restrictions on shape parameters. Please see the `scipy.stats.truncnorm` documentation for details.

In [27]:
from sklearn.utils.metaestimators import if_delegate_has_method

ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (/Users/pameladiaz/anaconda3/envs/MachineLearning/lib/python3.8/site-packages/sklearn/utils/metaestimators.py)

In [26]:
import eli5

# Assuming model is your trained gradient boosting model
eli5.show_weights(halving_search_gb, feature_names=X_prueba.columns)

ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (/Users/pameladiaz/anaconda3/envs/MachineLearning/lib/python3.8/site-packages/sklearn/utils/metaestimators.py)

In [29]:
from pdpbox import pdp, get_dataset

# Assuming model is your trained gradient boosting model
pdp_dist = pdp.pdp_isolate(halving_search_gb, X_prueba, 'feature_of_interest')
pdp.pdp_plot(pdp_dist, 'feature_of_interest')

ImportError: cannot import name 'get_dataset' from 'pdpbox' (/Users/pameladiaz/anaconda3/envs/MachineLearning/lib/python3.8/site-packages/pdpbox/__init__.py)